# Predicting Electricity Infrastructure Induced Wildfire Risk in California


Created by: Mengqi Yao [Last updated 06/27/22]

This is the main notebook for the risk models built in the paper. 

## PGE Data Sources

The following data sources are used (also cited) in this work. These are primarily data provided by PG&E. 

| Name of Data | Link | Instructions |
|:-:|:-:|:-:|
|**Feeder data**| [ICA Map Geospatial data](https://www.pge.com/b2b/distribution-resource-planning/integration-capacity-map.shtml)|You may need to make a PGE account to access the map. Then in the top right hand corner, click on the 'Download Spatial Data' button. The ICADisplay.gdb geodatabase should be downloaded.| 
|**Infrastructure component characteristics**| [WMP Data](https://pspsinfo.ss.pge.com/rr/artifacts/wildfire-mitigation-data/Attachment-6-GIS-Files.zip)|Including transformer, support structure (pole), and overhead conductor features. Download the 'Attachment 6: GIS Files' .zip file. Unzip locally. There should be 2 folders inside the main Attachments folder. Unzip the 'EDGIS2-12.gdb' folder. You may need to unzip the subfolders inside the EDGIS2-12 folder.  |
|**Fire incidents 2014-2019**| [WMP Data](https://www.pge.com/pge_global/common/pdfs/safety/emergency-preparedness/natural-disaster/wildfires/wildfire-mitigation-plan/SDR.zip)| Unzip, in the SDR Attachments, you will find IGNITIONS.gdb |
|**Wire Down events 2015-2019**| [WMP Data](https://www.pge.com/pge_global/common/pdfs/safety/emergency-preparedness/natural-disaster/wildfires/wildfire-mitigation-plan/SDR.zip)| In the SDR Spreadsheet.xlsx|
|**Additional conductor characteristics**| [WMP Data](https://www.pge.com/pge_global/common/pdfs/safety/emergency-preparedness/natural-disaster/wildfires/wildfire-mitigation-plan/reference-docs/MGRA007.zip)| More details on conductor|


## Aggregated feeder and Transmission line data

We merged the weather and vegetation data with the infrastructure data using feeder circuit IDs or transmission line names. Each record of the resulting combined dataset represents all features of one feeder or transmission line on a calendar day. We also recorded the total number of historical ignitions and wire-down events that happened on that circuit prior to that day.  We added binary fields indicating whether or not an ignition or wire-down happened on the day in question.  We relate ignition and wire-down events to the weather on the day when the event happened.

Feeder: https://drive.google.com/file/d/14IdZV7llwn5qp3uf8vIXtMU_AOVjoPXu/view?usp=sharing

Transmission line: https://drive.google.com/file/d/17XwtHXBp34hK2dvK2JMLPDY7fEJUQQk3/view?usp=sharing

In [1]:
# import packages
import geopandas as gpd
import pandas as pd
import numpy as np
from netCDF4 import Dataset
import re
import datetime
from numpy import mean

# plot library
import matplotlib.pyplot as plt
import matplotlib as mpl
import fiona
import seaborn as sns
import folium
import collections 
import math

import pickle
from tqdm.notebook import tqdm as tqdm


In [2]:
# ML packages
from datetime import timedelta
from openpyxl import load_workbook

import sklearn

from sklearn.metrics import accuracy_score,balanced_accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import cohen_kappa_score
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler

from sklearn import metrics
from sklearn import preprocessing

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

from sklearn.metrics import make_scorer

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve

from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_validate

from sklearn.ensemble import AdaBoostClassifier


from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint
from sklearn.calibration import CalibratedClassifierCV

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.pipeline import make_pipeline

from sklearn.svm import SVC
from sklearn.calibration import calibration_curve
from tqdm.notebook import tqdm as tqdm
from sklearn.metrics import brier_score_loss
import lightgbm as lgb


# Feeder Risk Model

In [3]:
data_raw = pd.read_feather('feeder_daily.ftr')
data_raw.columns

Index(['Day', 'Year', 'Time Period', 'FeederID', 'VOLTNUM', 'Total Miles',
       'Tier 2 Miles Proportion', 'Tier 3 Miles Proportion',
       'Zone 1 Miles Proportion', 'HFTDMILES Proportion',
       'Primary Conductor OH Proportion', 'Average Transformers Age',
       'Transformer Climate Zone_R', 'Transformer Climate Zone_S',
       'Transformer Climate Zone_T', 'Transformer Climate Zone_UNK',
       'Transformer Climate Zone_X', 'Average Support Structure Age',
       'Support Structure Material_ALM', 'Support Structure Material_CNCT',
       'Support Structure Material_CNTR', 'Support Structure Material_FIBR',
       'Support Structure Material_GUYP', 'Support Structure Material_LS',
       'Support Structure Material_OTH', 'Support Structure Material_PUSH',
       'Support Structure Material_S', 'Support Structure Material_STEL',
       'Support Structure Material_THBR', 'Support Structure Material_TREE',
       'Support Structure Material_UNK', 'Support Structure Material_WOOD',

In [4]:
def plot_roc(name, labels, predictions, **kwargs):
    fp, tp, _ = sklearn.metrics.roc_curve(labels, predictions)

    plt.plot(100*fp, 100*tp, label=name, linewidth=2, **kwargs)
    plt.xlabel('False positives [%]')
    plt.ylabel('True positives [%]')
    plt.xlim([-0.5,100])
    plt.ylim([0,100.5])
    plt.grid(True)
    ax = plt.gca()
    ax.set_aspect('equal')
    
    return fp, tp

In [5]:
# my own recall function

def p_recall_scroe(y_true, y_pred):
    score = recall_score(y_true, y_pred, average=None)
    final_score = score[1]
    return final_score

def n_recall_scroe(y_true, y_pred):
    score = recall_score(y_true, y_pred, average=None)
    final_score = score[0]
    return final_score

# cross validation
def CV_score(kfold, X, y, model):
    p_recall = []
    n_recall = []
    auc_score = []
    for train_ix, test_ix in tqdm(kfold.split(X, y)):
        # select rows
        train_X, test_X = X[train_ix], X[test_ix]
        train_y, test_y = y[train_ix], y[test_ix]
        
       
        clf = model.fit(train_X, train_y)
        y_hat = clf.predict(test_X)
        y_prob = clf.predict_proba(test_X)[:,1]
        #train_0, train_1 = len(train_y[train_y==0]), len(train_y[train_y==1])
        #test_0, test_1 = len(test_y[test_y==0]), len(test_y[test_y==1])
        #print('>Train: 0=%d, 1=%d, Test: 0=%d, 1=%d' % (train_0, train_1, test_0, test_1))    

        p_recall.append(p_recall_scroe(test_y, y_hat))
        n_recall.append(n_recall_scroe(test_y, y_hat))
        auc_score.append(roc_auc_score(test_y, y_prob))


    print('>--------final p recall-----------<')
    print('>Mean: %f' % (mean(p_recall)))
    print('>STD: %f' % (np.std(p_recall)))
    print('>--------final n recall-----------<')
    print('>Mean: %f' % (mean(n_recall)))
    print('>STD: %f' % (np.std(n_recall)))
    print('>--------final ROC-----------<')
    print('>Mean: %f' % (mean(auc_score)))
    print('>STD: %f' % (np.std(auc_score)))

## Logistic Regression

In [9]:
df = data_raw.copy()

# we take wildfire season in 2019 as the test data
mask_test = (df['Year']  == 2019) & (df['Week']  > 21) & (df['Week']  < 49)
test_df = df[mask_test]

# split by year
mask = (df['Year'] <2019) 

train_df = df[mask]
test_save = test_df[['FeederID','Year','Day','Ignitions']]

train_df = train_df.drop(columns =['Year','Day','Month', 'Week','Time Period','FeederID','Wire Down','Filtered Wire Down','Ignitions Caused by Wire Downs','Vegetation Contact Ignitions','Equipment Failure Ignitions']) 
test_df = test_df.drop(columns =['Year','Day','Month', 'Week','Time Period','FeederID','Wire Down','Filtered Wire Down','Ignitions Caused by Wire Downs','Vegetation Contact Ignitions','Equipment Failure Ignitions']) 
train_df.columns

Index(['VOLTNUM', 'Total Miles', 'Tier 2 Miles Proportion',
       'Tier 3 Miles Proportion', 'Zone 1 Miles Proportion',
       'HFTDMILES Proportion', 'Primary Conductor OH Proportion',
       'Average Transformers Age', 'Transformer Climate Zone_R',
       'Transformer Climate Zone_S', 'Transformer Climate Zone_T',
       'Transformer Climate Zone_UNK', 'Transformer Climate Zone_X',
       'Average Support Structure Age', 'Support Structure Material_ALM',
       'Support Structure Material_CNCT', 'Support Structure Material_CNTR',
       'Support Structure Material_FIBR', 'Support Structure Material_GUYP',
       'Support Structure Material_LS', 'Support Structure Material_OTH',
       'Support Structure Material_PUSH', 'Support Structure Material_S',
       'Support Structure Material_STEL', 'Support Structure Material_THBR',
       'Support Structure Material_TREE', 'Support Structure Material_UNK',
       'Support Structure Material_WOOD', 'Support Structure Material_WOST',
      

In [10]:
# Form np arrays of labels and features.
train_labels = np.array(train_df.pop('Ignitions'))
bool_train_labels = train_labels != 0
test_labels = np.array(test_df.pop('Ignitions'))

train_features = np.array(train_df)
test_features = np.array(test_df)

# standarlize data
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)
test_features_scaled = scaler.transform(test_features)

train_features_scaled = np.clip(train_features_scaled, -5, 5)
test_features_scaled = np.clip(test_features_scaled, -5, 5)

print('Training labels shape:', train_labels.shape)
print('Test labels shape:', test_labels.shape)

print('Training features shape:', train_features_scaled.shape)
print('Test features shape:', test_features_scaled.shape)

Training labels shape: (1981502,)
Test labels shape: (283811,)
Training features shape: (1981502, 65)
Test features shape: (283811, 65)


In [12]:
#Plain LR model
lgm = linear_model.LogisticRegression(fit_intercept=True, solver = 'liblinear', max_iter=10000,random_state = 42,C=0.0001)
lgm.fit(train_features_scaled, train_labels)
test_prob = lgm.predict_proba(test_features_scaled)[:,1]
print(roc_auc_score(test_labels, test_prob))


0.66755051571957


### Under-sampling

In [11]:
lgm_us = linear_model.LogisticRegression(fit_intercept=True, solver = 'liblinear', max_iter=10000,random_state = 42,C=0.0001)

undersample = RandomUnderSampler(sampling_strategy='majority')
X_resample, y_resample = undersample.fit_resample(train_features_scaled, train_labels)
lgm_us.fit(X_resample, y_resample)


test_prob = lgm_us.predict_proba(test_features_scaled)[:,1]
print(roc_auc_score(test_labels, test_prob))

0.7343651152382984


### Over-sampling

In [14]:
lgm_os = linear_model.LogisticRegression(fit_intercept=True, solver = 'liblinear', max_iter=10000,random_state = 42,C=0.0001)


bool_train_labels = train_labels != 0

pos_features = train_features_scaled[bool_train_labels]
neg_features = train_features_scaled[~bool_train_labels]

pos_labels = train_labels[bool_train_labels]
neg_labels = train_labels[~bool_train_labels]

ids = np.arange(len(pos_features))
choices = np.random.choice(ids, len(neg_features))


res_pos_features = pos_features[choices]
res_pos_labels = pos_labels[choices]

resampled_features = np.concatenate([res_pos_features, neg_features], axis=0)
resampled_labels = np.concatenate([res_pos_labels, neg_labels], axis=0)

order = np.arange(len(resampled_labels))
np.random.shuffle(order)
X_resample = resampled_features[order]
y_resample = resampled_labels[order]

lgm_os.fit(X_resample, y_resample)

test_prob = lgm_os.predict_proba(test_features_scaled)[:,1]
print(roc_auc_score(test_labels, test_prob))

0.7559070932311038


### SMOTE

In [17]:
lgm_SMOTE = linear_model.LogisticRegression(fit_intercept=True, solver = 'liblinear', max_iter=10000,random_state = 42,C=0.0001)
over = SMOTE(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=1)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)

# transform the dataset
X_resample, y_resample = pipeline.fit_resample(train_features_scaled, train_labels)

lgm_SMOTE.fit(X_resample,y_resample)

test_prob = lgm_SMOTE.predict_proba(test_features_scaled)[:,1]
print(roc_auc_score(test_labels, test_prob))

0.7555044811043905


### Blanced Weight

In [16]:
lgm_bw = linear_model.LogisticRegression(fit_intercept=True, solver = 'liblinear', max_iter=10000,class_weight='balanced',random_state = 42,C=0.0001)
lgm_bw.fit(train_features_scaled, train_labels)
test_prob = lgm_bw.predict_proba(test_features_scaled)[:,1]
print(roc_auc_score(test_labels, test_prob))

0.756676501038205


## Random Forest

In [18]:
df = data_raw.copy()

mask_test = (df['Year']  == 2019) & (df['Week']  > 21) & (df['Week']  < 49)
test_df = df[mask_test]

# split by year
mask = (df['Year'] <2019) 

train_df = df[mask]


train_df = train_df.drop(columns =['Year','Day','Month', 'Week','Time Period','FeederID','Wire Down','Filtered Wire Down','Ignitions Caused by Wire Downs','Vegetation Contact Ignitions','Equipment Failure Ignitions']) 
test_df = test_df.drop(columns =['Year','Day','Month', 'Week','Time Period','FeederID','Wire Down','Filtered Wire Down','Ignitions Caused by Wire Downs','Vegetation Contact Ignitions','Equipment Failure Ignitions']) 
train_df.columns

# Form np arrays of labels and features.
train_labels = np.array(train_df.pop('Ignitions'))
bool_train_labels = train_labels != 0
test_labels = np.array(test_df.pop('Ignitions'))

train_features = np.array(train_df)
test_features = np.array(test_df)


print('Training labels shape:', train_labels.shape)
print('Test labels shape:', test_labels.shape)

print('Training features shape:', train_features.shape)
print('Test features shape:', test_features.shape)

Training labels shape: (1981502,)
Test labels shape: (283811,)
Training features shape: (1981502, 65)
Test features shape: (283811, 65)


In [20]:
rf = RandomForestClassifier(n_estimators =100, 
                               max_features = 'sqrt',
                               min_samples_split= 3500,
                               min_samples_leaf = 10, 
                               bootstrap = 'True', 
                               random_state = 42)
rf.fit(train_features, train_labels)
test_prob = rf.predict_proba(test_features)[:,1]
print(roc_auc_score(test_labels, test_prob))

0.7565781051629482


### Balanced Weight

In [19]:
rf_BW = RandomForestClassifier(n_estimators =100, 
                               max_features = 'sqrt',
                               min_samples_split= 3500,
                               min_samples_leaf = 10, 
                               bootstrap = 'True', 
                               random_state = 42,
                               class_weight='balanced')

rf_BW.fit(train_features, train_labels)
test_prob = rf_BW.predict_proba(test_features)[:,1]
print(roc_auc_score(test_labels, test_prob))

0.761450110546642


### SMOTE

In [21]:
over = SMOTE(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=1)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)

# transform the dataset
X_resample, y_resample = pipeline.fit_resample(train_features, train_labels)

rf.fit(X_resample,y_resample)

test_prob = rf.predict_proba(test_features)[:,1]
print(roc_auc_score(test_labels, test_prob))

0.7354004638850751


## HGB model

In [22]:
df = data_raw.copy()

mask_test = (df['Year']  == 2019) & (df['Week']  > 21) & (df['Week']  < 49)
test_df = df[mask_test]

# split by year
mask = (df['Year'] <2019) 

train_df = df[mask]


train_df = train_df.drop(columns =['Year','Day','Month', 'Week','Time Period','FeederID','Wire Down','Filtered Wire Down','Ignitions Caused by Wire Downs','Vegetation Contact Ignitions','Equipment Failure Ignitions']) 
test_df = test_df.drop(columns =['Year','Day','Month', 'Week','Time Period','FeederID','Wire Down','Filtered Wire Down','Ignitions Caused by Wire Downs','Vegetation Contact Ignitions','Equipment Failure Ignitions']) 
train_df.columns

# Form np arrays of labels and features.
train_labels = np.array(train_df.pop('Ignitions'))
bool_train_labels = train_labels != 0
test_labels = np.array(test_df.pop('Ignitions'))

train_features = np.array(train_df)
test_features = np.array(test_df)


print('Training labels shape:', train_labels.shape)
print('Test labels shape:', test_labels.shape)

print('Training features shape:', train_features.shape)
print('Test features shape:', test_features.shape)

Training labels shape: (1981502,)
Test labels shape: (283811,)
Training features shape: (1981502, 65)
Test features shape: (283811, 65)


In [23]:
clf = lgb.LGBMClassifier(boosting_type='gbdt',
                         learning_rate = 0.3,
                         max_bin=63,
                         n_estimators=150,
                         min_data_in_leaf = 40,
                         max_depth = 2)

bbc = BalancedBaggingClassifier(base_estimator = clf,
                                sampling_strategy = 'majority',
                                random_state=42)
bbc.fit(train_features,train_labels)
test_prob = bbc.predict_proba(test_features)[:,1]
print(roc_auc_score( test_labels,test_prob))

[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=40
0.7755520538893953


# Transmission Line Model

In [24]:
data_raw = pd.read_feather('TL_daily.ftr')
data_raw.columns

Index(['index', 'Year', 'Day', 'max_wind_speed', 'max_gust_speed',
       'max_air_tempature', 'min_relative_humidify', 'mean_wind_speed: mean',
       'Date', 'Stations', 'T-Line Name', 'kV', 'Length', 'Wire Down',
       'Cause Category', 'Cause Details', 'BI Max', 'ERC Max', 'ETR Min',
       'FM1000 Min', 'FM100 Min', 'PET Min', 'PR Min', 'RH Min', 'SPH Min',
       'SRAD Max', 'TMAX Max', 'VPD Max', 'WS Max', 'CanopyHt_Mean',
       'CanopyHt_Max', 'min_dis', 'mean_dis', 'Hist WD Count', 'Week'],
      dtype='object')

In [73]:
df = data_raw.copy()

mask_test = (df['Year']  == 2019) 
test_df = df[mask_test]

# split by year
mask = (df['Year'] <2019) 

train_df = df[mask]


train_df = train_df.drop(columns =['Year','Day', 'Week','index','Date', 'Stations', 'T-Line Name','Cause Category', 'Cause Details']) 
test_df = test_df.drop(columns =['Year','Day', 'Week','index','Date', 'Stations', 'T-Line Name','Cause Category', 'Cause Details']) 
print(train_df.columns)

# Form np arrays of labels and features.
train_labels = np.array(train_df.pop('Wire Down'))
bool_train_labels = train_labels != 0
test_labels = np.array(test_df.pop('Wire Down'))

train_features = np.array(train_df)
test_features = np.array(test_df)


print('Training labels shape:', train_labels.shape)
print('Test labels shape:', test_labels.shape)

print('Training features shape:', train_features.shape)
print('Test features shape:', test_features.shape)

Index(['max_wind_speed', 'max_gust_speed', 'max_air_tempature',
       'min_relative_humidify', 'mean_wind_speed: mean', 'kV', 'Length',
       'Wire Down', 'BI Max', 'ERC Max', 'ETR Min', 'FM1000 Min', 'FM100 Min',
       'PET Min', 'PR Min', 'RH Min', 'SPH Min', 'SRAD Max', 'TMAX Max',
       'VPD Max', 'WS Max', 'CanopyHt_Mean', 'CanopyHt_Max', 'min_dis',
       'mean_dis', 'Hist WD Count'],
      dtype='object')
Training labels shape: (91097,)
Test labels shape: (23237,)
Training features shape: (91097, 25)
Test features shape: (23237, 25)


In [ ]:
clf = lgb.LGBMClassifier(boosting_type='gbdt',
                         importance_type='gain',
                         learning_rate = 0.1,
                         max_bin=15,
                         n_estimators=10,
                         min_data_in_leaf = 40,
                         max_depth = 2)

bbc = BalancedBaggingClassifier(base_estimator = clf,
                                sampling_strategy = 'majority',
                                random_state=42)

bbc.fit(train_features,train_labels)

roc_auc_score(test_labels, bbc.predict_proba(test_features)[:,1])


In [75]:
rf_BW = RandomForestClassifier(n_estimators =100, 
                               max_features = 'sqrt',
                               min_samples_split= 10,
                               min_samples_leaf = 14, 
                               bootstrap = 'True', 
                               random_state = 42,
                               class_weight='balanced')
rf_BW.fit(train_features,train_labels)

roc_auc_score(test_labels, rf_BW.predict_proba(test_features)[:,1])

0.7866259645643833